In [1]:
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
import glob
import numpy as np
import os

drive.mount('/content/drive')

%cd /content/drive/MyDrive/Sopheriods/Setup&tools/
from success_functions import *

%cd /content/drive/MyDrive/Sopheriods/ExperimentSpheroidsHT29/
!ls

Mounted at /content/drive
/content/drive/MyDrive/Sopheriods/Setup&tools
/content/drive/MyDrive/Sopheriods/ExperimentSpheroidsHT29
011223	051223	071223	111223	131223	151223	dapi	pi    pi_mask_3
041223	061223	091223	121223	141223	bf	dapi_m	pi_m


# First part: Method design and test

In [9]:
#@title Literally just a threshold
def custom_binary_thresh (img, thresh = 245): # pancreas: 183
  return cv2.threshold(255-img, thresh, 255, cv2.THRESH_BINARY)[1]

In [10]:
#@title Otsu + histogram equalization

def otsu_equalize (bf) :
  # take the negative of the image
  bf_neg = 255 - bf

  bf_neg = cv2.GaussianBlur(bf_neg, (5,5),0)

  (T, threshInv) = cv2.threshold(bf_neg, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  threshInv = cv2.bitwise_and(bf_neg, bf_neg, mask=threshInv)

  # increase the contrast
  alpha = 0.7
  threshInv = np.clip(alpha*threshInv, 0, 255).astype(np.uint8)
  threshInv = cv2.equalizeHist(threshInv) # now equalize the histogram that has got narrowed by a factor alpha

  (T, bf_final) = cv2.threshold(threshInv, 100 , 255, cv2.THRESH_BINARY)
  #(T, bf_final) = cv2.threshold(bf_neg, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  return bf_final



In [4]:
#@title Kmeans

def kmeans (bf, K = 2) :
  # take the negative of the image
  #bf_neg = 255 - bf
  bf_neg = bf

  z = np.float32(bf_neg.reshape((-1,1)))
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5, 1.0)
  compactness,labels,center = cv2.kmeans(z , K, None, criteria, 5, cv2.KMEANS_RANDOM_CENTERS)
  center = np.uint8(center)
  res = center[labels.flatten()]
  bf_final = res.reshape((bf_neg.shape))

  #bf_final = cv2.threshold(bf_final, 150, 255, cv2.THRESH_BINARY)[1]

  return bf_final

In [11]:
#@title Watershed
def watershed(bf):

  bf_neg = 255 - bf
  (T, threshInv) = cv2.threshold(bf_neg, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  #threshInv = cv2.bitwise_and(img[i], img[i], mask=threshInv)


  kernel = np.ones((3,3),np.uint8)
  opening = cv2.morphologyEx(threshInv,cv2.MORPH_OPEN,kernel, iterations = 2)
  opening = cv2.dilate(opening,kernel, iterations = 2) # probably results improve when we play with this

  # Finding sure foreground area
  dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3) # probably results improve when we play with this
  ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0) # probably results improve when we play with this
  # Finding unknown region
  sure_fg = np.uint8(sure_fg)
  unknown = cv2.subtract(opening,sure_fg)

  # Marker labelling
  ret, markers = cv2.connectedComponents(sure_fg)
  # Add one to all labels so that sure background is not 0, but 1
  markers = markers+1
  # Now, mark the region of unknown with zero
  markers[unknown==255] = 0


  bf_neg = cv2.cvtColor(bf_neg,cv2.COLOR_GRAY2RGB)
  markers = cv2.watershed(bf_neg,markers)


  res = 255*(markers == 2)

  #res = np.logical_and(bf_neg, res)

  return res



In [12]:
#@title Find contours

def contours(bf):
  bf_neg = 255 - bf

  #ret, thresh = cv2.threshold(bf_neg, 127, 255, cv2.THRESH_BINARY)
  (T, threshInv) = cv2.threshold(bf_neg, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  contours, hierarchy = cv2.findContours(threshInv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  #contour = max(contours, key = len) # take the largest contour.

  #bf_neg = cv2.cvtColor(bf_neg,cv2.COLOR_GRAY2RGB)
  #cv2.drawContours(bf_neg, contour, -1, (0,255,0), 3)
  bf_neg = cv2.fillPoly(np.zeros(bf_neg.shape), contours, 255)
  return bf_neg

In [13]:
#@title Water Means
def water_means(img):
  w = watershed(img).astype(np.uint8)
  wi = cv2.bitwise_and(255-img, 255-img, mask = w)
  ki = kmeans(wi, K = 3)
  return 255*(ki == np.max(ki))

In [8]:
try_method(otsu_equalize)

Output hidden; open in https://colab.research.google.com to view.

# Second part: evaluate methods quickly

In [14]:
#@title Take average similarity

def jaccard(img0, img1): return np.logical_and(img0, img1).sum()/np.logical_or(img0, img1).sum()

def assess_method_binary (method, dapi = True, pi = False, ht29 = True, pancreas = True):

  if dapi==0 and pi == 0: raise Exception('both solution masks set to false, cannot evaluate')
  if dapi==1 and pi == 1: raise Exception('both solution masks set to true, cannot evaluate')

  match_arr = []
  path_list = []

  base_path = "/content/drive/MyDrive/Sopheriods/"

  if ht29: path_list.append(f"{base_path}ExperimentSpheroidsHT29")
  if pancreas: path_list.append(f"{base_path}ExperimentPancreas")

  for path in path_list:
    for day in next(os.walk(path))[1]: # you may want to replace the "." string with a particular path directory!
      for folder in next(os.walk(f"{path}/{day}"))[1]:  # all folders in this folder
          try:
              # read image options
              opt = cv2.IMREAD_GRAYSCALE

              # read the 3 images of the same sphreoid
              bf = [cv2.imread(file, opt) for file in glob.glob(f"{path}/{day}/{folder}/bf*")][0]
              bf = cv2.resize(bf, (256, 256))

              # read mask
              mask = [cv2.imread(file, opt) for file in glob.glob(f"{path}/{day}/{folder}/dapi_m.JPG")][0] if dapi else [cv2.imread(file, opt) for file in glob.glob(f"{path}/{day}/{folder}/pi_m.JPG")][0]
              mask = cv2.resize(mask, (256, 256))

              # apply the method of segmentation
              bf_final = method(bf)

              # extract the similarity
              match_arr.append(jaccard(bf_final, mask))

          except Exception as e:
              print(f"Error processing folder: {folder}")
              print(e)

  # average performance
  return np.average(match_arr)

In [ ]:
methods = {custom_binary_thresh: 'Normal Threshold',
           otsu_equalize: 'Adaptative Threshold',
           kmeans: 'K-Means' ,
           contours : 'Find Contours',
           watershed : 'Watershed',
           water_means: "K-Shed"}


for m in methods:
  print(methods[m]+' : '+str(assess_method_binary(m, pi = False, ht29 = False)))

# Third part: debug some methods

In [ ]:
#@title Load images

dapi = 0 #@param {type: "boolean"}
pi = True #@param {type: "boolean"}
ht29 = True #@param {type: "boolean"}
pancreas = True #@param {type: "boolean"}

or_path = "/content/drive/MyDrive/Sopheriods/"
opt = cv2.IMREAD_GRAYSCALE

if dapi==0 and pi == 0: raise Exception('both solution masks set to false, cannot evaluate')
if dapi==1 and pi == 1: raise Exception('both solution masks set to true, cannot evaluate')

eval = "dapi_mask" if dapi else "pi_mask"

match_arr = []
path_list = []
imgs = []
masks = []

if ht29:
    path_list.append(f"{or_path}ExperimentSpheroidsHT29")
if pancreas:
    path_list.append(f"{or_path}ExperimentPancreas")

for path in path_list:
    for img in os.listdir(f"{path}/imgs/"):
        imgs.append(cv2.resize(cv2.imread(f"{path}/imgs/{img}", opt), (256, 256)))
    for mask in os.listdir(f"{path}/{eval}/"):
        masks.append(cv2.resize(cv2.imread(f"{path}/{eval}/{mask}", opt), (256, 256)))

In [ ]:
#@title Try several thresholds
for t in range(150, 255):
  match_arr = []
  for i in range(len(imgs)):
    #cv2_imshow(np.concatenate((imgs[i], masks[i]), axis=1))
    match_arr.append(jaccard(custom_binary_thresh(imgs[i], t),masks[i]))

  print(t, " ", np.average(match_arr))